In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys, MufexTestKeys
from quantfreedom.plotting.plotting_base import *
from quantfreedom.nb_funcs.nb_indicators.nb_indicators import *
from quantfreedom.exchanges.exchange import Exchange
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
from my_stuff import BybitTestKeys
from quantfreedom.exchanges.bybit_exchange.bybit import Bybit
from quantfreedom.enums import AccountState, OrderResult


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [20]:
@njit(cache=True)
def decrease_position(
    average_entry: float,
    bar_index: int,
    dos_index: int,
    equity: float,
    exit_fee_pct: float,
    exit_price: float,
    ind_set_index: int,
    market_fee_pct: float,
    order_status: int,
    position_size_asset: float,
    timestamp: int,
) -> tuple[AccountState, OrderResult]:
    pnl = 30
    fee_open = position_size_asset * average_entry * market_fee_pct  # math checked
    fee_close = position_size_asset * exit_price * exit_fee_pct  # math checked
    fees_paid = fee_open + fee_close  # math checked
    realized_pnl = round(pnl - fees_paid, 3)  # math checked

    # Setting new equity
    equity = round(realized_pnl + equity, 3)

    account_state = AccountState(
        # where we are at
        ind_set_index=ind_set_index,
        dos_index=dos_index,
        bar_index=bar_index,
        timestamp=timestamp,
        # account info
        available_balance=equity,
        cash_borrowed=0.0,
        cash_used=0.0,
        equity=equity,
        fees_paid=fees_paid,
        possible_loss=0.0,
        realized_pnl=realized_pnl,
        total_trades=0,
    )
    order_result = OrderResult(
        average_entry=0.0,
        can_move_sl_to_be=False,
        entry_price=0.0,
        entry_size_asset=0.0,
        entry_size_usd=0.0,
        exit_price=exit_price,
        leverage=0.0,
        liq_price=0.0,
        order_status=order_status,
        position_size_asset=0.0,
        position_size_usd=0.0,
        sl_pct=0.0,
        sl_price=0.0,
        tp_pct=0.0,
        tp_price=0.0,
    )

    return account_state, order_result

In [23]:
@njit(cache=True)
def testing():
    account_state: AccountState = AccountState(
        # where we are at
        ind_set_index=-1,
        dos_index=-1,
        bar_index=-1,
        timestamp=-1,
        # account info
        available_balance=1000,
        cash_borrowed=0.0,
        cash_used=0.0,
        equity=1000,
        fees_paid=0.0,
        possible_loss=0.0,
        realized_pnl=0.0,
        total_trades=0,
    )
    order_result: OrderResult = OrderResult(
        average_entry=0.0,
        can_move_sl_to_be=False,
        entry_price=0.0,
        entry_size_asset=0.0,
        entry_size_usd=0.0,
        exit_price=0.0,
        leverage=1.0,
        liq_price=0.0,
        order_status=0,
        position_size_asset=0.0,
        position_size_usd=0.0,
        sl_pct=0.0,
        sl_price=0.0,
        tp_pct=0.0,
        tp_price=0.0,
    )
    bar_index = 0
    dos_index = 0
    market_fee_pct = .0009
    ind_set_index = 0
    account_state, order_result = decrease_position(
        average_entry=order_result.average_entry,
        bar_index=bar_index,
        dos_index=dos_index,
        equity=account_state.equity,
        exit_fee_pct=market_fee_pct,
        exit_price=order_result.sl_price,
        ind_set_index=ind_set_index,
        market_fee_pct=market_fee_pct,
        order_status=2,
        position_size_asset=order_result.position_size_asset,
        timestamp=int(candles[bar_index, 0]),
    )
    print("done")

In [24]:
testing()

done
